# *Pipeline de limpeza dos dados*

In [48]:
#Resolvendo números em faixas --> acessar valor e substituir pela média da faixa
def calcular_media_faixa(valor):
    if isinstance(valor, str) and '-' in valor:
        try:
            # Divide a string em duas partes usando o '-' como separador
            parte1, parte2 = valor.split('-')
            # Converte as duas partes para float e calcula a média
            media = (float(parte1) + float(parte2)) / 2
            return media
        except ValueError:
            # Se algo der errado na conversão, retorna nulo
            return np.nan
    # Se não for uma faixa, retorna o valor como está
    return valor

In [49]:
import pandas as pd
import numpy as np
import math

In [50]:
df = pd.read_csv('imoveis_zap_fortaleza.csv',sep=";")
df.head()

,Preco,Tamanho,Quartos,Banheiros,Vagas,Bairro
0,R$ 1.380.282,Tamanho do imóvel119 m²,Quantidade de quartos3,Quantidade de banheiros3,Quantidade de vagas de garagem2,Apartamento para comprar emEngenheiro Luciano ...
1,A partir deR$ 591.401,Tamanho do imóvel40-60 m²,Quantidade de quartos1-2,Quantidade de banheiros1-2,Quantidade de vagas de garagem1,"Apartamento para comprar emAldeota, Fortaleza"
2,R$ 1.300.000,Tamanho do imóvel129 m²,Quantidade de quartos3,Quantidade de banheiros4,Quantidade de vagas de garagem3,"Apartamento para comprar emAldeota, Fortaleza"
3,R$ 635.000,Tamanho do imóvel73 m²,Quantidade de quartos3,Quantidade de banheiros2,Quantidade de vagas de garagem2,"Apartamento para comprar emPresidente Kennedy,..."
4,R$ 499.000,Tamanho do imóvel91 m²,Quantidade de quartos3,Quantidade de banheiros3,Quantidade de vagas de garagem2,"Apartamento para comprar emCocó, Fortaleza"


In [51]:
df_limpo = df.copy()

In [52]:
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1770 entries, 0 to 1769
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Preco      1770 non-null   object
 1   Tamanho    1770 non-null   object
 2   Quartos    1770 non-null   object
 3   Banheiros  1770 non-null   object
 4   Vagas      1770 non-null   object
 5   Bairro     1770 non-null   object
dtypes: object(6)
memory usage: 83.1+ KB


## *Coluna Preço*

In [53]:
df_limpo['Preco'] = df_limpo['Preco'].str.replace(r'[^\d.]', '', regex=True) # r'[^\d.]'Remove tudo que NÃO é dígito (\d) ou ponto (.).
df_limpo['Preco'] = df_limpo['Preco'].str.replace('.','')
#Converte a coluna para um tipo numérico /  O 'errors='coerce'':se um valor não puder ser convertido como uma string vazia, ele será transformado em NaN (Not a Number)
df_limpo['Preco'] = pd.to_numeric(df_limpo['Preco'],errors='coerce')


## *Coluna Tamanho*

In [54]:
df_limpo['Tamanho'] = df_limpo['Tamanho'].str.replace('Tamanho do imóvel','').str.replace('m²','')

In [55]:
#Aplicação da função à coluna 'Tamanho' original
df_limpo['Tamanho_Limpo'] = df_limpo['Tamanho'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Tamanho_Limpo'] = pd.to_numeric(df_limpo['Tamanho_Limpo'], errors='coerce')

## *Coluna Quartos*

In [56]:
df_limpo['Quartos'] = df_limpo['Quartos'].str.replace('Quantidade de quartos','')

In [57]:
#Aplicação da função à coluna 'Quartos' original --> porém será arrendado para o maior valor
df_limpo['Quartos_limpo'] = df_limpo['Quartos'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Quartos_limpo'] = pd.to_numeric(df_limpo['Quartos_limpo'], errors='coerce')
df_limpo['Quartos_limpo'] = df_limpo['Quartos_limpo'].apply(lambda x: math.ceil(x) if pd.notna(x) else x) #Aplicando arredondamento

## *Coluna Banheiros*

In [58]:
df_limpo['Banheiros'] = df_limpo['Banheiros'].str.replace('Quantidade de banheiros','')

In [59]:
#Aplicação da função à coluna 'Quartos' original --> porém será arrendado para o maior valor
df_limpo['Banheiros_limpo'] = df_limpo['Banheiros'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Banheiros_limpo'] = pd.to_numeric(df_limpo['Banheiros_limpo'], errors='coerce')
df_limpo['Banheiros_limpo'] = df_limpo['Banheiros_limpo'].apply(lambda x: math.ceil(x) if pd.notna(x) else x) #Aplicando arredondamento

## *Coluna Vagas*

In [60]:
df_limpo['Vagas'] = df_limpo['Vagas'].str.replace('Quantidade de vagas de garagem','')

In [62]:
#Aplicação da função à coluna 'Quartos' original --> porém será arrendado para o maior valor
df_limpo['Vagas_limpo'] = df_limpo['Vagas'].apply(calcular_media_faixa)
#Coluna seja numérica
df_limpo['Vagas_limpo'] = pd.to_numeric(df_limpo['Vagas_limpo'], errors='coerce')
df_limpo['Vagas_limpo'] = df_limpo['Vagas_limpo'].apply(lambda x: math.ceil(x) if pd.notna(x) else x) #Aplicando arredondamento

## *Coluna Bairro*

In [63]:
df_limpo['Bairro'] = df_limpo['Bairro'].str.replace('Apartamento para comprar em','').str.replace(', Fortaleza','')

## *Dropando Colunas de apoio e renomeando*

In [64]:
#Drop das colunas
df_limpo = df_limpo.drop(['Tamanho','Quartos','Banheiros','Vagas'],axis=1)

In [65]:
#Renomeando colunas
df_limpo.rename(columns={'Tamanho_Limpo': 'tamanho (m²)',
                         'Quartos_limpo':'qtd_quartos',
                         'Banheiros_limpo':'qtd_banheiro',
                         'Vagas_limpo':'qtd_vagas'},
                           inplace=True)

# *Salvando base de dados final*

In [66]:
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1770 entries, 0 to 1769
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Preco         1611 non-null   float64
 1   Bairro        1770 non-null   object 
 2   tamanho (m²)  1770 non-null   float64
 3   qtd_quartos   1770 non-null   int64  
 4   qtd_banheiro  1770 non-null   int64  
 5   qtd_vagas     1711 non-null   float64
dtypes: float64(3), int64(2), object(1)
memory usage: 83.1+ KB


In [67]:
#Salvando em arquivo CSV
nome_do_arquivo = 'imoveis_zap_fortaleza_ready.csv'
df_limpo.to_csv(nome_do_arquivo, index=False, sep=';', encoding='utf-8-sig')